# Simple Qubit Flipping Example

This example explains how we can flip one or more qubits using cirq

In [ ]:
import cirq
import numpy as np
from cirq_iqm.iqm_sampler import IQMSampler

To connect to Helmi backend we pass the cocos url as an argument to the IQMSampler  

In [ ]:
sampler = IQMSampler("https://qc.vtt.fi/cocos")

## Defining our Functions

First we define a function that helps us calculate the probability of measuring a desired state

In [ ]:
def calculate_success_probability(counts: dict, shots: int, desired_state: str) -> float:
    """
    Calculate the success probability from the job results.
    """
    success_counts = counts.get(desired_state, 0)
    return success_counts / shots

Next we create a helper function to convert the measured bits into a suitable format

In [ ]:
def fold_func(x: np.ndarray) -> str:
    """Fold the measured bit arrays into strings."""
    return ''.join(map(lambda x: chr(x + ord('0')), x))

We can now create our circuits. The `single_flip_circuit` applies an X gate to a single qubit to flip it from |0> to |1> and returns the correct mapping.

In [ ]:
def single_flip_circuit(qubit: int):
    qubit = cirq.NamedQubit(f'QB{qubit}')
    circuit = cirq.Circuit()
    circuit.append(cirq.X(qubit))
    circuit.append(cirq.measure(qubit, key='M'))
    return circuit

The `flip_circuit` function creates a quantum circuit with X-gates applied to every qubit

In [ ]:
def flip_circuit(qubits: list[int]):
    qubits = [cirq.NamedQubit(f'QB{i}') for i in qubits]
    circuit = cirq.Circuit()
    for qubit in qubits:
        circuit.append(cirq.X(qubit))
    circuit.append(cirq.measure(*qubits, key='M'))
    return circuit

This is a helper function for measuring our circuit

In [ ]:
def measure_circuit(circuits, shots: int, desired_state: str):
    for i, circuit in enumerate(circuits):
        result = sampler.run(circuit, repetitions=shots)
        counts = result.histogram(key='M', fold_func=fold_func)
        success_probability = calculate_success_probability(
                counts, shots, desired_state,
            )
        print("\nCounts:", counts)
        print(f"Success probability: {success_probability * 100:.2f}%")

`flip_single_qubit` and `flip_qubits` are functions to measure our circuits

In [ ]:
def flip_single_qubit(qubits: str, desired_state: str = '1', shots: int = 1000) -> None:
    circuits = []
    print("\nFlip one qubit at a time\n")
    for qb in qubits:
        circuit = single_flip_circuit(qb)
        circuits.append(circuit)
    return measure_circuit(circuits, shots, desired_state)

In [ ]:
flip_single_qubit('1', 2000)

In [ ]:
def flip_qubits(qubits: list[int]| None = None, desired_state: str = '11111', shots: int = 1000) -> None:
    circuits = []
    print("\nFlip all qubits \n")
    qubits_list = qubits or list(range(1, 6))
    print(qubits_list)
    circuit = flip_circuit(qubits_list)
    circuits.append(circuit)
    return measure_circuit(circuits, shots, desired_state)

In [ ]:
flip_qubits([1, 2, 3, 4, 5])

## Summary

In this notebook we have demonstrated how we can flip one or more bits on Helmi using cirq-on-iqm.

